In [1]:
#To hide warnings export PYTHONWARNINGS="ignore"
#Imports{

import os
from os.path import dirname
from os.path import join
os.environ['KERAS_BACKEND'] = 'theano'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Cha


import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import cPickle as pickle
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.misc import imread
from scipy.misc import imresize
from scipy.io import loadmat

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import keras
from keras import backend as K
K.set_image_dim_ordering('th')
#}

Using Theano backend.


Model architecture
<img src="../../data/misc/vgg16.png">

In [3]:
#Code snippet needed to read activation values from each layer of the pre-trained artificial neural networks
def get_activations(model, layer, X_batch):
    #keras.backend.function(inputs, outputs, updates=None)
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    #The learning phase flag is a bool tensor (0 = test, 1 = train)
    activations = get_activations([X_batch,0])
    return activations

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0         
__________

In [5]:
#Function to pre-process the input image to ensure uniform size and color
def preprocess_image_batch(image_paths, img_size=None, crop_size=None, color_mode='rgb', out=None):
    """
    Consistent preprocessing of images batches
    :param image_paths: iterable: images to process
    :param crop_size: tuple: crop images if specified
    :param img_size: tuple: resize images if specified
    :param color_mode: Use rgb or change to bgr mode based on type of model you want to use
    :param out: append output to this iterable if specified
    """
    img_list = []

    for im_path in image_paths:
        img = imread(im_path, mode='RGB')
        #print im_path
        #print img.shape
        if img_size:
            img = imresize(img, img_size)

        img = img.astype('float32')
        # We normalize the colors (in RGB space) with the empirical means on the training set
        img[:, :, 0] -= 123.68
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 103.939
        # We permute the colors to get them in the BGR order
        if color_mode == 'bgr':
            img[:, :, [0, 1, 2]] = img[:, :, [2, 1, 0]]
        img = img.transpose((2, 0, 1))

        if crop_size:
            img = img[:, (img_size[0] - crop_size[0]) // 2:(img_size[0] + crop_size[0]) // 2
            , (img_size[1] - crop_size[1]) // 2:(img_size[1] + crop_size[1]) // 2]

        img_list.append(img)

    try:
        img_batch = np.stack(img_list, axis=0)
    except:
        print im_path
        raise ValueError('when img_size and crop_size are None, images'
                ' in image_paths must have the same shapes.')

    if out is not None and hasattr(out, 'append'):
        out.append(img_batch)
    else:
        return img_batch

In [10]:
#Function to predict the top 5 accuracy
def top5accuracy(true, predicted):
    assert len(true) == len(predicted)
    result = []
    flag  = 0
    for i in range(len(true)):
        flag  = 0
        temp = true[i]
        for j in predicted[i]:
            if j == temp:
                flag = 1
                break
        if flag == 1:
            result.append(1)
        else:
            result.append(0)
    counter = 0.
    for i in result:
        if i == 1:
            counter += 1.
    error = 1.0 - counter/float(len(result))
    #print len(np.where(np.asarray(result) == 1)[0])
    return len(np.where(np.asarray(result) == 1)[0]), error

In [6]:
#Load the details of all the 1000 classes and the function to conver the synset id to words{
meta_clsloc_file = '../../data/meta_clsloc.mat'
synsets = loadmat(meta_clsloc_file)['synsets'][0]
synsets_imagenet_sorted = sorted([(int(s[0]), str(s[1][0])) for s in synsets[:1000]],key=lambda v: v[1])
corr = {}
for j in range(1000):
    corr[synsets_imagenet_sorted[j][0]] = j

corr_inv = {}
for j in range(1, 1001):
    corr_inv[corr[j]] = j

def id_to_words(id_):
    return synsets[corr_inv[id_] - 1][2][0]

def pprint_output(out, n_max_synsets=10):
    wids = []
    best_ids = out.argsort()[::-1][:10]
    for u in best_ids:
        wids.append(str(synsets[corr_inv[u] - 1][1][0]))
    #print('%.2f' % round(100 * out[u], 2) + ' : ' + id_to_words(u)+' '+ str(synsets[corr_inv[u] - 1][1][0]))
    return wids

In [7]:
#Code snippet to load the ground truth labels to measure the performance{
truth = {}
with open('../../data/ILSVRC2014_clsloc_validation_ground_truth.txt') as f:
    line_num = 1
    for line in f.readlines():
        ind_ = int(line)
        temp  = None
        for i in synsets_imagenet_sorted:
            if i[0] == ind_:
                temp = i
        #print ind_,temp
        if temp != None:
            truth[line_num] = temp
        else:
            print '##########', ind_
            pass
        line_num += 1
#}

In [28]:
# Loading the folder to be procesed from command line{
p = 'inanimate'
tmp = p.replace('/','_')
print tmp


p_num = 1
url_path = '../../data/'+p+'/'
#}

inanimate


In [29]:
# Prepare the image list and pre-process them{
true_wids = []
im_list = []
for i in os.listdir(url_path):
    if not i.startswith('~') and not i.startswith('.'):
        #print i, truth
        temp = i.split('.')[0].split('_')[2]
        true_wids.append(truth[int(temp)][1])
        im_list.append(url_path+i)

im = preprocess_image_batch(im_list,img_size=(256,256), crop_size=(224,224), color_mode="rgb")
#}


In [30]:
# Model parmeters and running the model from the loaded weights{
model_name = 'VGG16'
model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
#KFold
k = 4

im_train, im_test = train_test_split(im_list, test_size=0.2, random_state=234)
#}

In [31]:
#Training data pkl
fp_name = '../../data/pkl/'+str(p)+'_train_no_lesion_performance.txt'
fp = open(fp_name,'a+')
im_temp = preprocess_image_batch(im_train,img_size=(256,256), crop_size=(224,224), color_mode="rgb")
out = model.predict(im_temp,batch_size=64)

true_valid_wids = []
for i in im_train:
        temp1 = i.split('/')[4]
        temp = temp1.split('.')[0].split('_')[2]
        true_valid_wids.append(truth[int(temp)][1])

predicted_valid_wids = []
for i in range(len(im_train)):
    #print im_list[i], pprint_output(out[i]), true_wids[i]
    predicted_valid_wids.append(pprint_output(out[i]))

count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

fp.write(str(p)+' '+str(count)+' '+str(len(im_train))+' '+str(error)+'\n')


print len(true_valid_wids), len(predicted_valid_wids), len(im_train)
print count, error


#}
# Code snippet to get the activation values and saving information{
data = np.array([])

i = 0
result ={}
for layer in model.layers:
    weights = layer.get_weights()
    if len(weights) > 0:
        activations = get_activations(model,i,im_temp)
        if result.get(layer.name, None) is None:
            result[layer.name] = activations[0]
            temp = np.mean(activations[0], axis=0).ravel()
            if layer.name != 'predictions':
                print layer.name,len(weights),len(activations), activations[0].shape, np.mean(activations[0], axis=0).shape, temp.shape
                data = np.append(data, temp)
    i += 1

fp.close()
with open('../../data/pkl/'+str(p)+'_train_'+model_name+'.pkl', 'wb') as f:
    pickle.dump(data, f)

155 155 155
134 0.135483870968
block1_conv1 2 1 (155, 64, 224, 224) (64, 224, 224) (3211264,)
block1_conv2 2 1 (155, 64, 224, 224) (64, 224, 224) (3211264,)
block2_conv1 2 1 (155, 128, 112, 112) (128, 112, 112) (1605632,)
block2_conv2 2 1 (155, 128, 112, 112) (128, 112, 112) (1605632,)
block3_conv1 2 1 (155, 256, 56, 56) (256, 56, 56) (802816,)
block3_conv2 2 1 (155, 256, 56, 56) (256, 56, 56) (802816,)
block3_conv3 2 1 (155, 256, 56, 56) (256, 56, 56) (802816,)
block4_conv1 2 1 (155, 512, 28, 28) (512, 28, 28) (401408,)
block4_conv2 2 1 (155, 512, 28, 28) (512, 28, 28) (401408,)
block4_conv3 2 1 (155, 512, 28, 28) (512, 28, 28) (401408,)
block5_conv1 2 1 (155, 512, 14, 14) (512, 14, 14) (100352,)
block5_conv2 2 1 (155, 512, 14, 14) (512, 14, 14) (100352,)
block5_conv3 2 1 (155, 512, 14, 14) (512, 14, 14) (100352,)
fc1 2 1 (155, 4096) (4096,) (4096,)
fc2 2 1 (155, 4096) (4096,) (4096,)


In [32]:
#Testing data pkl
fp_name = '../../data/pkl/'+str(p)+'_test_no_lesion_performance.txt'
fp = open(fp_name,'a+')

im_temp = preprocess_image_batch(im_test,img_size=(256,256), crop_size=(224,224), color_mode="rgb")
out = model.predict(im_temp,batch_size=64)

true_valid_wids = []
for i in im_test:
        temp1 = i.split('/')[4]
        temp = temp1.split('.')[0].split('_')[2]
        true_valid_wids.append(truth[int(temp)][1])

predicted_valid_wids = []
for i in range(len(im_test)):
    #print im_list[i], pprint_output(out[i]), true_wids[i]
    predicted_valid_wids.append(pprint_output(out[i]))

count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)

fp.write(str(p)+' '+str(count)+' '+str(len(im_test))+' '+str(error)+'\n')


print len(true_valid_wids), len(predicted_valid_wids), len(im_test)
print count, error


#}
# Code snippet to get the activation values and saving information{
data = np.array([])

i = 0
result ={}
for layer in model.layers:
    weights = layer.get_weights()
    if len(weights) > 0:
        activations = get_activations(model,i,im_temp)
        if result.get(layer.name, None) is None:
            result[layer.name] = activations[0]
            temp = np.mean(activations[0], axis=0).ravel()
            if layer.name != 'predictions':
                print layer.name,len(weights),len(activations), activations[0].shape, np.mean(activations[0], axis=0).shape, temp.shape
                data = np.append(data, temp)
    i += 1

fp.close()
with open('../../data/pkl/'+str(p)+'_test_'+model_name+'.pkl', 'wb') as f:
    pickle.dump(data, f)

39 39 39
29 0.25641025641
block1_conv1 2 1 (39, 64, 224, 224) (64, 224, 224) (3211264,)
block1_conv2 2 1 (39, 64, 224, 224) (64, 224, 224) (3211264,)
block2_conv1 2 1 (39, 128, 112, 112) (128, 112, 112) (1605632,)
block2_conv2 2 1 (39, 128, 112, 112) (128, 112, 112) (1605632,)
block3_conv1 2 1 (39, 256, 56, 56) (256, 56, 56) (802816,)
block3_conv2 2 1 (39, 256, 56, 56) (256, 56, 56) (802816,)
block3_conv3 2 1 (39, 256, 56, 56) (256, 56, 56) (802816,)
block4_conv1 2 1 (39, 512, 28, 28) (512, 28, 28) (401408,)
block4_conv2 2 1 (39, 512, 28, 28) (512, 28, 28) (401408,)
block4_conv3 2 1 (39, 512, 28, 28) (512, 28, 28) (401408,)
block5_conv1 2 1 (39, 512, 14, 14) (512, 14, 14) (100352,)
block5_conv2 2 1 (39, 512, 14, 14) (512, 14, 14) (100352,)
block5_conv3 2 1 (39, 512, 14, 14) (512, 14, 14) (100352,)
fc1 2 1 (39, 4096) (4096,) (4096,)
fc2 2 1 (39, 4096) (4096,) (4096,)


In [33]:
out_r = []

image_list_test = '../../data/pkl/'+p+'_image_list_test.txt'
with open(image_list_test,'wb') as f:
    for i in im_test:
        f.write(i+'\n')

kf = KFold(n_splits= k)
fold = 1
fp_name = '../../data/pkl/'+str(p)+'_no_lesion_performance.txt'
fp = open(fp_name,'a+')
for train_index, valid_index in kf.split(im_train):
    print("Starting Fold: ", fold)
    im_valid_train = [im_train[i] for i in train_index] 
    im_valid_test = [im_train[i] for i in valid_index]
    
    image_list_train = '../../data/pkl/'+p+'_image_list_train_fold_'+str(fold)+'.txt'
    with open(image_list_train,'wb') as f:
        for i in im_valid_train:
            f.write(i+'\n')
    
    image_list_valid = '../../data/pkl/'+p+'_image_list_valid_fold_'+str(fold)+'.txt'
    with open(image_list_valid,'wb') as f:
        for i in im_valid_test:
            f.write(i+'\n')
    
   
    im_temp = preprocess_image_batch(im_valid_test,img_size=(256,256), crop_size=(224,224), color_mode="rgb")
    out = model.predict(im_temp,batch_size=64)

    true_valid_wids = []
    for i in im_valid_test:
            temp1 = i.split('/')[4]
            temp = temp1.split('.')[0].split('_')[2]
            true_valid_wids.append(truth[int(temp)][1])
    
    predicted_valid_wids = []
    for i in range(len(im_valid_test)):
        #print im_list[i], pprint_output(out[i]), true_wids[i]
        predicted_valid_wids.append(pprint_output(out[i]))
        
    count, error  = top5accuracy(true_valid_wids, predicted_valid_wids)
    
    fp.write(str(p)+' '+str(fold)+' '+str(count)+' '+str(len(im_valid_test))+' '+str(error)+'\n')

    
    print len(true_valid_wids), len(predicted_valid_wids), len(im_valid_test)
    print count, error
    
    
    #}
    # Code snippet to get the activation values and saving information{
    data = np.array([])

    i = 0
    result ={}
    for layer in model.layers:
        weights = layer.get_weights()
        if len(weights) > 0:
            activations = get_activations(model,i,im_temp)
            if result.get(layer.name, None) is None:
                result[layer.name] = activations[0]
                temp = np.mean(activations[0], axis=0).ravel()
                if layer.name != 'dense_3':
                    print layer.name,len(weights),len(activations), activations[0].shape, np.mean(activations[0], axis=0).shape, temp.shape
                    data = np.append(data, temp)
        i += 1
    print data.shape
    out_r.append(data)
    fold += 1
    
fp.close()

('Starting Fold: ', 1)
39 39 39
34 0.128205128205
block1_conv1 2 1 (39, 64, 224, 224) (64, 224, 224) (3211264,)
block1_conv2 2 1 (39, 64, 224, 224) (64, 224, 224) (3211264,)
block2_conv1 2 1 (39, 128, 112, 112) (128, 112, 112) (1605632,)
block2_conv2 2 1 (39, 128, 112, 112) (128, 112, 112) (1605632,)
block3_conv1 2 1 (39, 256, 56, 56) (256, 56, 56) (802816,)
block3_conv2 2 1 (39, 256, 56, 56) (256, 56, 56) (802816,)
block3_conv3 2 1 (39, 256, 56, 56) (256, 56, 56) (802816,)
block4_conv1 2 1 (39, 512, 28, 28) (512, 28, 28) (401408,)
block4_conv2 2 1 (39, 512, 28, 28) (512, 28, 28) (401408,)
block4_conv3 2 1 (39, 512, 28, 28) (512, 28, 28) (401408,)
block5_conv1 2 1 (39, 512, 14, 14) (512, 14, 14) (100352,)
block5_conv2 2 1 (39, 512, 14, 14) (512, 14, 14) (100352,)
block5_conv3 2 1 (39, 512, 14, 14) (512, 14, 14) (100352,)
fc1 2 1 (39, 4096) (4096,) (4096,)
fc2 2 1 (39, 4096) (4096,) (4096,)
predictions 2 1 (39, 1000) (1000,) (1000,)
(13556712,)
('Starting Fold: ', 2)
39 39 39
32 0.17948

In [34]:
#Saving all the data into pkl files
for i in range(k):
    name = p+'_fold_'+str(i+1)+'_train_'+model_name
    out_data = out_r[i]
    with open('../../data/pkl/'+name+'.pkl', 'wb') as f:
        pickle.dump(out_data, f)
    print(name, len(out_data))

('inanimate_fold_1_train_VGG16', 13556712)
('inanimate_fold_2_train_VGG16', 13556712)
('inanimate_fold_3_train_VGG16', 13556712)
('inanimate_fold_4_train_VGG16', 13556712)
